## Загрузка данных
#### Данные беру из Human_learning

# Colab

In [1]:
! gdown --id 1P3aYu5kdMLOVxgl2K6wjzx-HtWB5J_wu

"gdown" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [2]:
! unzip / content / uk_used_cars.zip -d / content / uk_used_cars /

"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [11]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/uk_used_cars/bmw.csv')

In [5]:
df.head()

NameError: name 'df' is not defined

# Pycharm

In [1]:
from zipfile import ZipFile

with ZipFile('uk_used_cars.zip', 'r') as zf:
    zf.extractall()

In [1]:
import pandas as pd

In [97]:
df = pd.read_csv(r'C:\PyCharm\Python\Data_Science_GM\4.Human_learning\bmw.csv')

In [98]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


In [99]:
df.shape

(10781, 9)

## Обучающая и тестовая выборка (train and test)

In [100]:
from sklearn.model_selection import train_test_split

In [101]:
# train, test = train_test_split(df, train_size=0.7)

In [102]:
train, test = train_test_split(df, random_state=42, train_size=0.6)  # default train_size = 0.75

In [103]:
len(train), len(test), len(test + train), len(df)

(6468, 4313, 10781, 10781)

In [104]:
len(test) / len(df)

0.4000556534644282

## Валидационная выборка

##### Для получения валидационной выборки нужно разбить тестовую часть пополам, на валидационную и тестовую

In [105]:
val, test = train_test_split(test, random_state=42, train_size=0.5)

In [106]:
len(train) / len(df)

0.5999443465355718

In [107]:
len(val) / len(df)

0.19998144884519062

In [108]:
len(test) / len(df)

0.20007420461923756

## Список фичей

In [109]:
train.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [110]:
train.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5133,X6,2015,25000,Semi-Auto,26100,Diesel,205,45.6,3.0
2121,X2,2019,31980,Automatic,4804,Petrol,145,34.0,2.0
6576,X2,2018,22995,Semi-Auto,24136,Petrol,145,38.7,2.0
8330,3 Series,2016,12999,Manual,52224,Diesel,125,62.8,2.0
3161,X3,2019,32980,Semi-Auto,1961,Diesel,150,54.3,2.0


## Принцип преобразования категориальных фичей

##### Метод OneHotEncoding

In [111]:
train.transmission

5133    Semi-Auto
2121    Automatic
6576    Semi-Auto
8330       Manual
3161    Semi-Auto
          ...    
5734    Semi-Auto
5191    Semi-Auto
5390    Semi-Auto
860     Semi-Auto
7270    Automatic
Name: transmission, Length: 6468, dtype: object

In [112]:
pd.get_dummies(train['transmission'])

,Automatic,Manual,Semi-Auto
5133,0,0,1
2121,1,0,0
6576,0,0,1
8330,0,1,0
3161,0,0,1
...,...,...,...
5734,0,0,1
5191,0,0,1
5390,0,0,1
860,0,0,1


## Первый запуск Кэтбуста

In [113]:
from catboost import CatBoostRegressor

##### В первом запуске катбуста, поставим те же фичи, что были в хьюман лёрнинг и сравним результаты

In [114]:
all_features = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
     'mpg', 'engineSize']

X = ['year', 'transmission', 'engineSize']

cat_features = ['transmission']

y = ['price']

##### Catboost support metrics
https://catboost.ai/en/docs/references/custom-metric__supported-metrics

##### Создаём модель и настраиваем её параметры

In [115]:
model = CatBoostRegressor(cat_features=cat_features,
                          eval_metric='MAPE',
                          random_state=42,
                          verbose=100)
# verbose - выводить каждую n-ую итерацию

In [116]:
train[X].head()

,year,transmission,engineSize
5133,2015,Semi-Auto,3.0
2121,2019,Automatic,2.0
6576,2018,Semi-Auto,2.0
8330,2016,Manual,2.0
3161,2019,Semi-Auto,2.0


In [117]:
train[y].head()

,price
5133,25000
2121,31980
6576,22995
8330,12999
3161,32980


##### Обучение модели

In [118]:
model.fit(train[X], train[y], eval_set=(val[X], val[y]))

Learning rate set to 0.068263
0:	learn: 0.4603375	test: 0.4555509	best: 0.4555509 (0)	total: 18.1ms	remaining: 18.1s
100:	learn: 0.1546393	test: 0.1529687	best: 0.1529687 (100)	total: 1.7s	remaining: 15.1s
200:	learn: 0.1528213	test: 0.1517332	best: 0.1517332 (200)	total: 3.15s	remaining: 12.5s
300:	learn: 0.1517532	test: 0.1514386	best: 0.1514000 (288)	total: 4.75s	remaining: 11s
400:	learn: 0.1509722	test: 0.1515508	best: 0.1514000 (288)	total: 6.33s	remaining: 9.46s
500:	learn: 0.1502967	test: 0.1516166	best: 0.1514000 (288)	total: 7.96s	remaining: 7.93s
600:	learn: 0.1499412	test: 0.1516849	best: 0.1514000 (288)	total: 9.58s	remaining: 6.36s
700:	learn: 0.1494768	test: 0.1516928	best: 0.1514000 (288)	total: 11.2s	remaining: 4.79s
800:	learn: 0.1490851	test: 0.1518297	best: 0.1514000 (288)	total: 12.8s	remaining: 3.18s
900:	learn: 0.1487173	test: 0.1519119	best: 0.1514000 (288)	total: 14.4s	remaining: 1.58s
999:	learn: 0.1483869	test: 0.1519628	best: 0.1514000 (288)	total: 15.9s	rem

##### Ошибка катбуста MAPE: 0.15118
##### Моя ошибка в human learning на том же пресете (год, двигатель, трансмиссия) MAPE: 0.16037

##### Для подтверждения победы машинного обучения над человеческим проверим на тестовой выборке (уже проверили на валидационной)

In [119]:
model.predict(test[X])

array([45436.28801924, 21738.83525716, 16205.32627896, ...,
       28444.11570854, 16205.32627896, 27824.78614511])

In [120]:
test['price_pred'] = model.predict(test[X])

In [121]:
test.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,price_pred
4984,X6,2019,52990,Semi-Auto,3086,Diesel,145,34.9,3.0,45436.288019
4880,X1,2018,24081,Semi-Auto,13245,Diesel,150,60.1,2.0,21738.835257
9132,1 Series,2016,12999,Automatic,68949,Petrol,200,43.5,2.0,16205.326279
6451,1 Series,2020,11995,Semi-Auto,10,Petrol,150,34.5,2.0,32088.707415
7199,3 Series,2020,29875,Semi-Auto,150,Petrol,145,42.2,2.0,32088.707415


##### Импортируем библиотеку для расчёта ошибок

In [122]:
from sklearn.metrics import  mean_absolute_error, mean_absolute_percentage_error

In [123]:
def error(y_true, y_pred):
    print(mean_absolute_error(y_true, y_pred))
    print(mean_absolute_percentage_error(y_true, y_pred))

In [124]:
error(test.price, test.price_pred)

3366.679482256595
0.15363582890850797


#### Катбуст выдал лучший результат против Human learning:
#### catboost MAPE: 0.15166607293455014 | Human learning MAPE: 0.16037327734163415
#### catboost MAE: 3345.7246072100406   | Human learning MAE: 3777.5172798216277

## Обучение на всех фичах

In [125]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [126]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'random_seed': 42,
              'verbose':100}

In [127]:
model = CatBoostRegressor(**parameters)

In [128]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

Learning rate set to 0.068263
0:	learn: 0.4607438	test: 0.4563750	best: 0.4563750 (0)	total: 19.2ms	remaining: 19.2s
100:	learn: 0.0904302	test: 0.0896388	best: 0.0896388 (100)	total: 1.78s	remaining: 15.8s
200:	learn: 0.0781784	test: 0.0803125	best: 0.0803125 (200)	total: 3.55s	remaining: 14.1s
300:	learn: 0.0729925	test: 0.0764506	best: 0.0764506 (300)	total: 5.43s	remaining: 12.6s
400:	learn: 0.0695552	test: 0.0744676	best: 0.0744553 (399)	total: 7.31s	remaining: 10.9s
500:	learn: 0.0670683	test: 0.0732827	best: 0.0732827 (500)	total: 9.2s	remaining: 9.16s
600:	learn: 0.0652835	test: 0.0726170	best: 0.0726128 (597)	total: 11s	remaining: 7.28s
700:	learn: 0.0636320	test: 0.0720622	best: 0.0720500 (695)	total: 12.7s	remaining: 5.43s
800:	learn: 0.0623408	test: 0.0714586	best: 0.0714329 (794)	total: 14.7s	remaining: 3.64s
900:	learn: 0.0611487	test: 0.0711564	best: 0.0711321 (898)	total: 16.4s	remaining: 1.81s
999:	learn: 0.0601407	test: 0.0708439	best: 0.0708403 (998)	total: 18.2s	rem

##### На 3 фичах в Human learning лучший результат был 16.03%, а в Catboost на всех фичах 7.08%

##### Сделаем предикт на тесте

In [129]:
model.predict(test[X])

array([57417.53972205, 22003.56027391, 13461.40876567, ...,
       30553.87789981, 13818.32099473, 24849.36058457])

In [130]:
test['price_pred_all'] = model.predict(test[X])

In [131]:
test.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,price_pred,price_pred_all
4984,X6,2019,52990,Semi-Auto,3086,Diesel,145,34.9,3.0,45436.288019,57417.539722
4880,X1,2018,24081,Semi-Auto,13245,Diesel,150,60.1,2.0,21738.835257,22003.560274
9132,1 Series,2016,12999,Automatic,68949,Petrol,200,43.5,2.0,16205.326279,13461.408766
6451,1 Series,2020,11995,Semi-Auto,10,Petrol,150,34.5,2.0,32088.707415,32149.002131
7199,3 Series,2020,29875,Semi-Auto,150,Petrol,145,42.2,2.0,32088.707415,35100.139852


In [132]:
error(test.price, test.price_pred_all)

1556.5378327680894
0.07367840910148211


### НА всех фичах ошибка 7.36% в catboost

## Количество итераций и learning rate

##### Желательно, чтобы лучший результат достигался ближе к концу обучения, для более тщательного обучения, но если лучший результат приходится, на одну из последних итераций, то можно увеличить learning rate
##### Можно уменьшить learning rate, что бы модель училась не так быстро
##### НА 3 фичах:

In [139]:
X = ['year', 'transmission', 'engineSize']

cat_features = ['transmission']

y = ['price']

In [137]:
model = CatBoostRegressor(cat_features=cat_features,
                          learning_rate=0.03,
                          eval_metric='MAPE',
                          random_state=42,
                          verbose=100)
# verbose - выводить каждую n-ую итерацию

In [138]:
model.fit(train[X], train[y], eval_set=(val[X], val[y]))

0:	learn: 0.4746987	test: 0.4699730	best: 0.4699730 (0)	total: 23.5ms	remaining: 23.5s
100:	learn: 0.1681230	test: 0.1638970	best: 0.1638970 (100)	total: 1.84s	remaining: 16.4s
200:	learn: 0.1561398	test: 0.1538311	best: 0.1538311 (200)	total: 3.31s	remaining: 13.2s
300:	learn: 0.1543096	test: 0.1525317	best: 0.1525309 (299)	total: 4.67s	remaining: 10.8s
400:	learn: 0.1534465	test: 0.1519221	best: 0.1519198 (397)	total: 6.16s	remaining: 9.21s
500:	learn: 0.1527824	test: 0.1516330	best: 0.1516164 (480)	total: 7.79s	remaining: 7.75s
600:	learn: 0.1522590	test: 0.1514284	best: 0.1514284 (600)	total: 9.4s	remaining: 6.24s
700:	learn: 0.1518052	test: 0.1513704	best: 0.1513673 (689)	total: 11s	remaining: 4.7s
800:	learn: 0.1514495	test: 0.1513967	best: 0.1513651 (703)	total: 12.6s	remaining: 3.13s
900:	learn: 0.1510958	test: 0.1513657	best: 0.1513489 (873)	total: 14.3s	remaining: 1.57s
999:	learn: 0.1508297	test: 0.1513297	best: 0.1513168 (992)	total: 15.9s	remaining: 0us

bestTest = 0.15131

##### На всех фичах:

In [144]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [148]:
parameters = {'cat_features': cat_features,
              'learning_rate': 0.075,
              'eval_metric': 'MAPE',
              'random_seed': 42,
              'verbose':100}

In [149]:
model = CatBoostRegressor(**parameters)

In [150]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4582738	test: 0.4539380	best: 0.4539380 (0)	total: 18.2ms	remaining: 18.2s
100:	learn: 0.0876480	test: 0.0862924	best: 0.0862924 (100)	total: 1.91s	remaining: 17s
200:	learn: 0.0773759	test: 0.0790647	best: 0.0790404 (198)	total: 3.7s	remaining: 14.7s
300:	learn: 0.0719070	test: 0.0753211	best: 0.0753211 (300)	total: 5.54s	remaining: 12.9s
400:	learn: 0.0687633	test: 0.0735751	best: 0.0735751 (400)	total: 7.38s	remaining: 11s
500:	learn: 0.0664724	test: 0.0725579	best: 0.0725579 (500)	total: 9.21s	remaining: 9.17s
600:	learn: 0.0645872	test: 0.0717504	best: 0.0717504 (600)	total: 11s	remaining: 7.33s
700:	learn: 0.0627572	test: 0.0710084	best: 0.0710084 (700)	total: 12.9s	remaining: 5.49s
800:	learn: 0.0613750	test: 0.0704878	best: 0.0704878 (800)	total: 14.7s	remaining: 3.65s
900:	learn: 0.0599069	test: 0.0698996	best: 0.0698936 (896)	total: 16.5s	remaining: 1.82s
999:	learn: 0.0587290	test: 0.0697822	best: 0.0696893 (998)	total: 18.3s	remaining: 0us

bestTest = 0.06968926

##### Модель улучшила результат за счёт Learning rate

In [152]:
parameters = {'cat_features': cat_features,
              'learning_rate': 0.08,
              'eval_metric': 'MAPE',
              'random_seed': 42,
              'verbose':100}

In [153]:
model = CatBoostRegressor(**parameters)

In [154]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 26.4ms	remaining: 26.4s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 1.88s	remaining: 16.7s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 3.68s	remaining: 14.6s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 5.48s	remaining: 12.7s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 7.26s	remaining: 10.9s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 9.06s	remaining: 9.02s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 10.8s	remaining: 7.2s
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 12.6s	remaining: 5.38s
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 14.5s	remaining: 3.6s
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 16.3s	remaining: 1.79s
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 18.2s	remaining: 0us

bestTest = 0.068

#### Early_stopping_rounds - помогает не перебирать всю 1000 вариантов, а останваливать обучение, если не происходит улучшений в течении n итераций

In [155]:
X = ['year', 'transmission', 'engineSize']

cat_features = ['transmission']

y = ['price']

In [156]:
model = CatBoostRegressor(cat_features=cat_features,
                          early_stopping_rounds=200,
                          eval_metric='MAPE',
                          random_state=42,
                          verbose=100)
# verbose - выводить каждую n-ую итерацию

In [157]:
model.fit(train[X], train[y], eval_set=(val[X], val[y]))

Learning rate set to 0.068263
0:	learn: 0.4603375	test: 0.4555509	best: 0.4555509 (0)	total: 19.4ms	remaining: 19.4s
100:	learn: 0.1546393	test: 0.1529687	best: 0.1529687 (100)	total: 1.58s	remaining: 14.1s
200:	learn: 0.1528213	test: 0.1517332	best: 0.1517332 (200)	total: 2.9s	remaining: 11.5s
300:	learn: 0.1517532	test: 0.1514386	best: 0.1514000 (288)	total: 4.41s	remaining: 10.2s
400:	learn: 0.1509722	test: 0.1515508	best: 0.1514000 (288)	total: 5.92s	remaining: 8.85s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1514000297
bestIteration = 288

Shrink model to first 289 iterations.
